In [2]:
import pandas as pd
import os
import pickle

from model import Transformer # this is the transformer.py file
import torch
from torch import nn
import numpy as np

In [16]:
# Generate some dooky data

# temp should be 1 * 120 * 15
# precip should be 1 * 120 * 10


# Set random seed for reproducibility
np.random.seed(0)
# Generate input data
temp_input = np.random.randn(1, 120, 15)
precip_input = np.random.randn(1, 120, 15)
combined_input = np.stack([temp_input, precip_input], axis=2)

# Generate target data with a smaller time dimension
temp_target = np.random.randn(1, 12, 15)
precip_target = np.random.randn(1, 12, 15)
combined_target = np.stack([temp_target, precip_target], axis=2)

combined_input_tensor = torch.tensor(combined_input, dtype=torch.float32)
combined_target_tensor = torch.tensor(combined_target, dtype=torch.float32)

In [17]:
# Hyperparameters

d_temp = 15
d_precip = 15
d_model = 15
ffn_hidden = 2048
num_heads = 5
drop_prob = 0.1
num_layers = 1

transformer = Transformer(d_model, d_temp, d_precip, ffn_hidden, num_heads, drop_prob, num_layers)


output = transformer(combined_input_tensor, combined_target_tensor)

TypeError: 'float' object cannot be interpreted as an integer